In [ ]:
import os
import time
import numpy as np
import tensorflow as tf


In [ ]:
path_to_file = tf.keras.utils.get_file( "shakespeare.txt", "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt", )

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


In [ ]:
print(text[:250])   ### The first 250 characters in text


First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [ ]:
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")   ### To see how many unique characters are in our corpus/document.

65 unique characters


# Process the text


Vectorize the text

In [ ]:
example_texts = ["abcdefg", "xyz"]
# TODO 1
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

In [ ]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [ ]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)                 ##This layer recovers the characters from the vectors of IDs, and returns them as a tf.RaggedTensor of characters:



In [ ]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [ ]:
tf.strings.reduce_join(chars, axis=-1).numpy()  ###join the characters back to string


array([b'abcdefg', b'xyz'], dtype=object)

In [ ]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [ ]:
# TODO 2
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [ ]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)


In [ ]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i
t
i


In [ ]:
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

In [ ]:
sequences = ids_dataset.batch(seq_length + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [ ]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [ ]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [ ]:
split_input_target(list("Tensorflow"))


(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [ ]:
dataset = sequences.map(split_input_target)


In [ ]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


Create training batches


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

#Build The Model


In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)
print(vocab_size)
# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

65


In [ ]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__()
        # TODO - Create an embedding layer
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        # TODO - Create a GRU layer
        self.gru = tf.keras.layers.GRU(
            rnn_units, return_sequences=True, return_state=True
        )
        # TODO - Finally connect it with a dense layer
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        # The following line was not indented properly
        x = self.embedding(inputs, training=training)

        # Get the batch size
        batch_size = tf.shape(inputs)[0]

        if states is None:
            # Initialize the state with the correct shape
            states = self.gru.get_initial_state(batch_size=batch_size)

        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x

In [ ]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [ ]:
embedding_dim

256

In [ ]:
rnn_units

1024

In [ ]:
dataset.take(1)

<_TakeDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()


Model: "my_model_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_11 (Embedding)             │ (64, 100, 256)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_11 (GRU)                         │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (64, 100, 66)               │          67,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [ ]:
sampled_indices


array([ 3, 60, 30, 27, 21, 37, 32, 30,  8, 56, 54, 41,  6,  8, 10, 27, 55,
       24, 45, 41,  3, 18, 45, 57, 45,  1, 51, 31, 31, 51, 55, 64, 39, 60,
       47, 33, 53, 36, 60, 45, 63,  1, 62,  3, 22, 28, 46, 24, 21, 29, 13,
        4, 33, 15, 29, 56,  8, 28,  4,  1, 63, 45, 56, 20, 47, 29,  7, 10,
       20, 51, 16, 15, 27, 64, 52, 17,  5, 64, 40, 10, 21, 44, 32, 20, 61,
       48, 60, 41, 37, 52,  6, 25, 15, 18,  6,  1, 62, 44, 47,  1])

In [ ]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b"in place.\n\nPost:\n'Tell him,' quoth she, 'my mourning weeds are done,\nAnd I am ready to put armour on"

Next Char Predictions:
 b"!uQNHXSQ-qob'-3NpKfb!Efrf\nlRRlpyZuhTnWufx\nw!IOgKHP?$TBPq-O$\nxfqGhP,3GlCBNymD&ya3HeSGviubXm'LBE'\nweh\n"


Train the model


In [ ]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)


In [ ]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1892962, shape=(), dtype=float32)


In [ ]:
tf.exp(example_batch_mean_loss).numpy()


65.97634

In [ ]:
model.compile(optimizer="adam", loss=loss)


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"
# Name of the checkpoint files
# Added '.weights.h5' to the checkpoint_prefix
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix, save_weights_only=True
)

In [ ]:
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])


Epoch 1/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1023s 6s/step - loss: 3.0596
Epoch 2/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1037s 6s/step - loss: 1.9170
Epoch 3/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 999s 6s/step - loss: 1.6271
Epoch 4/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1049s 6s/step - loss: 1.4795
Epoch 5/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1043s 6s/step - loss: 1.3930
Epoch 6/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1030s 6s/step - loss: 1.3303
Epoch 7/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1046s 6s/step - loss: 1.2796
Epoch 8/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1041s 6s/step - loss: 1.2346
Epoch 9/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 992s 6s/step - loss: 1.1934
Epoch 10/10
172/172 ━━━━━━━━━━━━━━━━━━━━ 1035s 6s/step - loss: 1.1489


In [ ]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature = temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

        # Create a mask to prevent "[UNK]" from being generated.
        skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
        sparse_mask = tf.SparseTensor(
            # Put a -inf at each bad index.
            values=[-float("inf")] * len(skip_ids),
            indices=skip_ids,
            # Match the shape to the vocabulary
            dense_shape=[len(ids_from_chars.get_vocabulary())],
        )
        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
        # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, "UTF-8")
        input_ids = self.ids_from_chars(input_chars).to_tensor()

        # Run the model.
        # predicted_logits.shape is [batch, char, next_char_logits]
        predicted_logits, states = self.model(
            inputs=input_ids, states=states, return_state=True
        )
        # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits / self.temperature
        # Apply the prediction mask: prevent "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

        # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)

        # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

        # Return the characters and model state.
        return predicted_chars, states

In [ ]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)


In [ ]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
print("\nRun time:", end - start)

ROMEO:
Small robe as a good to? Women, you warrant, king, where's my sister.

ISABELLA:
Whither? Hast yet daynigh, then darest not: ho,
Whose difteen need my brother Montague,
Since hearing so harth from the measure;
He hath closely to my very hath, did them not.
Must cure my life, and of your eyes for revenue,
I children withar; one courre I have hall:
Ay, and you shall pierce in his noble,--
ANTONIO:
How now I preteous chielded 'gid!

TRANIO:
What is it? sir? this god? thou hast rark'd you?

DUKE VINCENTIO:
I will be cold your prison.

ANGELO:
Now, hath a man doth say a montal's child, and that
I say, a word of newn. Hast thou, Warwick coming! Now, my lord,
By stain with thine easy my princely groats,
He is now prosperise o' the while: your absequon.

LARTIUS:
Hark, I created: milter,
Saferit, point you.' Nay, a monstrous answer;
but though 'tis countenhing ene.'
Kith glory, the king's give than a intinent.

THOMASES:
I begged my deed. And I am not for your comperage.

HASTINGS:
Spri

In [ ]:
start = time.time()
states = None
next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
result = [next_char]

for n in range(1000):
    next_char, states = one_step_model.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, "\n\n" + "_" * 80)
print("\nRun time:", end - start)

tf.Tensor(
[b"ROMEO:\nNay, none! but now to take him woman. Be not, long.\nTo-minder, I love to see your lace.\nThe third of my daughter King Richard?\nYou are flateous to begin and figst\nMy intending parts and gentlemen,\nMakes our audicles call take it me bown tigle,\nHis citizeness to the Duke of Naples\nWe'll be the bastle of thy mind:\nAll put their hatres; and, in the words are hither\nHe's all aught. I six on my head.\n\nPAgo: Here comes Messance; I'll be gone.\n\nJOHN OF GAUNT:\nIt is to save them to his close men.\nWhat news am I take up the controast?\nO, for what thou be preteceated bore than\nI have to't brankful blaze the nurted\nTickle and bitter burdy.\n\nServant:\nMeantily that you good wholes of this, love him up\nAnd think but while posting to the city behind\nMy nation deny more shrew that you should be a grace.\n\nQUEEN ELIZABETH:\nWhile we envying them, whose easies are nurse\ntrue, to think your will till you live to Faint: a sweetest\nprecious pitchpour lusty fi

Export the generator


In [ ]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [ ]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
Thou say, none as first loed of! Gentle king,
Where pretty time without mine: you are full of twate


Advanced: Customized Training


In [ ]:
class CustomTraining(MyModel):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {"loss": loss}

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [ ]:
model.fit(dataset, epochs=1)


172/172 ━━━━━━━━━━━━━━━━━━━━ 1006s 6s/step - loss: 2.4928


In [ ]:
import time

import tensorflow as tf

EPOCHS = 10


mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    # Use reset_state() instead of reset_states()
    mean.reset_state()
    for batch_n, (inp, target) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs["loss"])

        if batch_n % 50 == 0:
            template = (
                f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            )
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f"Epoch {epoch+1} Loss: {mean.result().numpy():.4f}")
    print(f"Time taken for 1 epoch {time.time() - start:.2f} sec")
    print("_" * 80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.0487
Epoch 1 Batch 50 Loss 1.9394
Epoch 1 Batch 100 Loss 1.7911
Epoch 1 Batch 150 Loss 1.7294

Epoch 1 Loss: 1.8335
Time taken for 1 epoch 1041.92 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7042
Epoch 2 Batch 50 Loss 1.6110
Epoch 2 Batch 100 Loss 1.5343
Epoch 2 Batch 150 Loss 1.5604

Epoch 2 Loss: 1.5929
Time taken for 1 epoch 1041.92 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.4831
Epoch 3 Batch 50 Loss 1.4531
Epoch 3 Batch 100 Loss 1.5023
Epoch 3 Batch 150 Loss 1.4387

Epoch 3 Loss: 1.4667
Time taken for 1 epoch 998.72 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.3768
Epoch 4 Batch 50 Loss 1.4006
Epoch 4 Batch 100 Loss 1.3997
Epoch 4 Batch 150 Loss 1.3731

Epoch 4 Loss: 1.3887
Time taken for 1 epoch 1041.91 sec
______________________________________________________________